In [ ]:
from pybring import BringApi
import requests
import pandas as pd
import numpy as np

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
# tuto https://towardsdatascience.com/how-to-integrate-google-sheets-and-jupyter-notebooks-c469309aacea

In [ ]:
#!ls

In [ ]:
spreadsheet_key = r'1aTqsmCbfN8moxBcSwIp1BMXEQDIKRTG9CXWDik51ZZI'

In [ ]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('bringapi-92345de4ed45.json', scope)
gc = gspread.authorize(credentials)

In [ ]:
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("Bring!")
table = worksheet.get_all_values()
table = np.array(table)

In [ ]:
table

In [ ]:
df = pd.DataFrame(data={'item': table[:, 0], 'quantity':  table[:, 1]})
df = df[df['item'] != ''].copy()
df['quantity'] = df['quantity'].astype(float)

In [ ]:
df['unit'] = df['item'].str.extract(r'(\(.+\))')
df['unit'] = df['unit'].str[1:-1].values

In [ ]:
n_i = []
for i, u in zip(df['item'].tolist(), df['unit'].tolist()):
    n_i.append(i.replace(' ({unit})'.format(unit=u), ''))
df['item'] = n_i
df

In [ ]:
reduced_df = df[df['quantity'] != 0]
reduced_df

In [ ]:
b = BringApi(uuid='mike.claure@gmail.com', bringuuid='bringMike2021', use_login=True)

In [ ]:
b.get_items()

In [ ]:
for index in range(len(reduced_df)):
    item_i = reduced_df.values[index][0]
    quantity_i = reduced_df.values[index][1]
    unit_i = reduced_df.values[index][2]
    
    b.purchase_item(item=item_i, specification='{} {}'.format(quantity_i, unit_i))